In [ ]:
###  INPUT ###

import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from datetime import datetime
from keras.models import Model
from keras.models import Sequential
from keras.layers import Input, Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from tensorflow.keras.optimizers import SGD,Adam,RMSprop

input_data = pd.read_csv('C:/Users/USER/Downloads/SAMPLE/SAMPLE/2023_smartFarm_AI_hackathon_dataset.csv')

arr= ['DBSF1059','DBSF1093','DBSF1172','DBSF1180','DBSF1279','DBSF1294','DBSF1385','DBSF1459','DBSF1469','DBSF1507','DBSF1539','DBSF1601','DBSF1621','DBSF1871']

input_data['date']= input_data['date'].astype(str)
input_data["year"]=input_data["date"].str[:4]
input_data["month"]=input_data["date"].str[4:6]
input_data["day"]=input_data["date"].str[6:]
input_data['date']=input_data["year"]+'-'+input_data["month"]+'-'+input_data["day"]
input_data=input_data.drop(['month','year','day'],axis=1)
input_data['date']=pd.to_datetime(input_data['date'])
input_data=input_data.set_index(keys=['date'],inplace=False,drop=True)


In [ ]:
data1 = input_data[input_data['frmDist'] == arr[0]]
for i in range(1,len(arr)):
    data = input_data[input_data['frmDist'] == arr[i]]
    tmp=data
    data_result=pd.concat([data1,tmp])
    data1= data_result

In [ ]:
input_data = input_data[input_data['HeatingEnergyUsage_cumsum'] != 0.000]
input_data = input_data[input_data['outtrn_cumsum'] != 0]

In [ ]:
data1 = input_data[input_data['frmDist'] == arr[0]]
for i in range(1,len(arr)):
    data = input_data[input_data['frmDist'] == arr[i]]
    tmp=data
    data_result=pd.concat([data1,tmp])
    data1= data_result

In [ ]:
data = data_result[['HeatingEnergyUsage_cumsum','ec','frtstCo','fcluHg']]
target =data_result['outtrn_cumsum']

In [ ]:
target = target.to_frame()

In [ ]:
scaler = MinMaxScaler()
scale_cols =0
scale_cols = ['HeatingEnergyUsage_cumsum','ec','frtstCo','fcluHg']
scaled = scaler.fit_transform(data[scale_cols])
df_min_max= pd.DataFrame(scaled, columns = ['HeatingEnergyUsage_cumsum','ec','frtstCo','fcluHg'])
df_min_max.index = data.index
data=df_min_max

In [ ]:
scale_cols = ['outtrn_cumsum']
scaled = scaler.fit_transform(target[scale_cols])
df_min_max= pd.DataFrame(scaled, columns = ['outtrn_cumsum'])
df_min_max.index = target.index
target=df_min_max

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2,shuffle=False)

In [ ]:
aDam = keras.optimizers.Adam(lr=0.01)

In [ ]:
model = Sequential()
model.add(Dense(64,input_dim=4, activation='relu'))
model.add(Dense(32,activation='sigmoid'))
model.add(Dense(8,activation='sigmoid'))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='mse', optimizer=aDam, metrics='mse')
model.summary()

In [ ]:
model.fit(X_train,y_train, epochs=300,batch_size=16, verbose=1)

In [ ]:
y_pred = model.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2score = r2_score(y_test, y_pred)

In [ ]:
print("RMSE:", rmse)
print("R2_score:", r2score)